In [113]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import concatenate
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers import Conv1D
from keras.layers import BatchNormalization

In [54]:
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [55]:
'''
category = []
label = []
train_category=[]
for i in range(df.shape[0]):
    category.append(df['Category'][i])
    label.append(df['Label'][i])
tokenizer = Tokenizer(nb_words=None)
tokenizer.fit_on_texts(category)
category_index = tokenizer.word_index

for i in range(df.shape[0]):
    train_category.append(category_index[df['Category'][i]])

label = np.array(label)
train_category = to_categorical(np.asarray(train_category))
'''
category = []
label = []
label_test = []
train_category=[]
test_category=[]
for i in range(df.shape[0]):
    category.append(df['Category'][i])
    label.append(df['Label'][i])
for j in range(df2.shape[0]):
    label_test.append(df2['Label'][j])
tokenizer = Tokenizer(nb_words=None)
tokenizer.fit_on_texts(category)
category_index = tokenizer.word_index
category_index.setdefault('living',23)
category_index.setdefault('middleeast',24)
category_index.setdefault('us',25)

for i in range(df.shape[0]):
    train_category.append(category_index[df['Category'][i]])
for j in range(df2.shape[0]):
    test_category.append(category_index[df2['Category'][j]])
label = np.array(label)
label_test = np.array(label_test)
label-=3
train_category = to_categorical(np.asarray(train_category))
padding = np.zeros((2040,3))
train_category = np.concatenate((train_category,padding),axis=1)
test_category = to_categorical(np.asarray(test_category))

C:\Users\USER\Anaconda3\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [56]:
embeddings_index = {}
f = open('glove.6B.100d.txt','r',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [57]:
sent = []
data = []
data_test = []
for i in range(df.shape[0]):
    sent=[(df['Headline'][i])]
    stopwords=[",","the"]
    list=[word for word in word_tokenize(sent[0]) if word not in stopwords]
    pos = nltk.pos_tag(list)
    wordnet_pos = []
    lemmatizer = WordNetLemmatizer()
    for word, tag in pos:
        if tag.startswith('J'):
            wordnet_pos.append(wordnet.ADJ)
        elif tag.startswith('V'):
            wordnet_pos.append(wordnet.VERB)
        elif tag.startswith('N'):
            wordnet_pos.append(wordnet.NOUN)
        elif tag.startswith('R'):
            wordnet_pos.append(wordnet.ADV)
        else:
            wordnet_pos.append(wordnet.NOUN)

    tokens = [lemmatizer.lemmatize(pos[n][0], pos=wordnet_pos[n]) for n in range(len(pos))]
    data.append(tokens)
    
for j in range(df2.shape[0]):
    sent=[(df2['Headline'][j])]
    stopwords=[",","the"]
    list=[word for word in word_tokenize(sent[0]) if word not in stopwords]
    pos = nltk.pos_tag(list)
    wordnet_pos = []
    lemmatizer = WordNetLemmatizer()
    for word, tag in pos:
        if tag.startswith('J'):
            wordnet_pos.append(wordnet.ADJ)
        elif tag.startswith('V'):
            wordnet_pos.append(wordnet.VERB)
        elif tag.startswith('N'):
            wordnet_pos.append(wordnet.NOUN)
        elif tag.startswith('R'):
            wordnet_pos.append(wordnet.ADV)
        else:
            wordnet_pos.append(wordnet.NOUN)

    tokens = [lemmatizer.lemmatize(pos[n][0], pos=wordnet_pos[n]) for n in range(len(pos))]
    data_test.append(tokens)

In [58]:
tokenizer_train = Tokenizer(nb_words=None)
tokenizer_test = Tokenizer(nb_words=None)
tokenizer_train.fit_on_texts(data)
tokenizer_test.fit_on_texts(data_test)

data_total = data + data_test
tokenizer_total = Tokenizer(nb_words=None)
tokenizer_total.fit_on_texts(data_total)

sequences = tokenizer_train.texts_to_sequences(data)
sequences_test = tokenizer_train.texts_to_sequences(data_test)

word_index = tokenizer_train.word_index
word_index_test = tokenizer_test.word_index
word_index_total = tokenizer_total.word_index
print('Found %s unique tokens.' % len(word_index))
print('Found %s unique tokens.' % len(word_index_test))
print('Found %s unique tokens.' % len(word_index_total))

data1 = pad_sequences(sequences, maxlen=40)
data_test1 = pad_sequences(sequences_test, maxlen=40)

Found 8753 unique tokens.
Found 1692 unique tokens.
Found 9225 unique tokens.


In [59]:
embedding_matrix = np.zeros((len(word_index_total) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index_total) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=40,
                            trainable=False)

In [60]:
nb_validation_samples = int(0.2 * data1.shape[0])
'''
x_train1 = data1[:-nb_validation_samples]
x_train2 = train_category[:-nb_validation_samples]
x_test1 = data_test1
x_test2 = test_category
y_train = label[:-nb_validation_samples]

x_val1 = data1[-nb_validation_samples:]
x_val2 = train_category[-nb_validation_samples:]
y_val = label[-nb_validation_samples:]
'''
x_train1 = data1
x_train2 = train_category
x_test1 = data_test1
x_test2 = test_category
y_train = label

In [149]:
sequence_input = Input(shape=(40,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x1 = Conv1D(filters=100, kernel_size=5,strides=1, padding="causal",activation="relu")(embedded_sequences)
x1 = Conv1D(filters=200, kernel_size=5,strides=1, padding="causal",activation="relu")(x1)
x1 = MaxPooling1D(pool_size=5, strides=1, padding='valid')(x1)
#x1 = Conv1D(filters=32, kernel_size=5,strides=1, padding="causal",activation="relu")(x1)
#x1 = Conv1D(filters=32, kernel_size=5,strides=1, padding="causal",activation="relu")(x1)
x1 = SpatialDropout1D(0.2)(x1)
x1 = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(x1)
category_input = Input(shape=(26,))
x = concatenate([x1,category_input],axis=-1)
x = BatchNormalization()(x)
x = Dense(256)(x)
x = LeakyReLU(0.2)(x)
x = BatchNormalization()(x)
x = Dense(256)(x)
x = LeakyReLU(0.2)(x)
output = Dense(1)(x)
model = Model([sequence_input,category_input],output)
model.compile(loss='mse',optimizer='adam')


In [150]:
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 40, 100)      922600      input_33[0][0]                   
__________________________________________________________________________________________________
conv1d_35 (Conv1D)              (None, 40, 100)      50100       embedding_2[11][0]               
__________________________________________________________________________________________________
conv1d_36 (Conv1D)              (None, 40, 200)      100200      conv1d_35[0][0]                  
___________________________________________________________________________________________

In [146]:
#history = model.fit([x_train1,x_train1,x_train2],y_train,shuffle= True,validation_data=([x_val1,x_val1,x_val2],y_val),epochs=4,batch_size=128)
history = model.fit([x_train1,x_train2],y_train,shuffle= True,validation_split=0.2,epochs=10,batch_size=128)

Train on 1632 samples, validate on 408 samples
Epoch 1/50
1632/1632 [==============================] - 7s 4ms/step - loss: 0.9592 - val_loss: 0.4975
Epoch 2/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.6943 - val_loss: 0.5368
Epoch 3/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.6412 - val_loss: 0.5773
Epoch 4/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.6096 - val_loss: 0.5485
Epoch 5/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.5819 - val_loss: 0.5189
Epoch 6/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.5889 - val_loss: 0.5557
Epoch 7/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.5817 - val_loss: 0.5092
Epoch 8/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.6010 - val_loss: 0.5756
Epoch 9/50
1632/1632 [==============================] - 5s 3ms/step - loss: 0.5796 - val_loss: 0.5054
Epoch 10/50
1632/1632 [============

In [147]:
#y_predict = model.predict([x_test1,x_test1,x_test2])
y_predict = model.predict([x_test1,x_test2])

In [148]:
y_predict+3


array([[2.607104 ],
       [2.4373333],
       [1.8779496],
       [2.7679307],
       [2.5950289],
       [2.7514975],
       [1.621383 ],
       [2.1831155],
       [2.8775349],
       [2.1246736],
       [3.326408 ],
       [2.8893075],
       [3.0848923],
       [2.7587354],
       [2.5588312],
       [2.9568   ],
       [2.8802156],
       [2.5639138],
       [2.7365332],
       [3.1580179],
       [2.33274  ],
       [3.1200695],
       [3.7010674],
       [2.9455743],
       [2.6477714],
       [2.5423355],
       [2.6827362],
       [2.510148 ],
       [2.9709907],
       [2.5781837],
       [2.312414 ],
       [2.665026 ],
       [2.8191102],
       [1.698108 ],
       [2.8154798],
       [2.0211797],
       [2.830543 ],
       [2.7645874],
       [2.7129095],
       [3.2562976],
       [2.882338 ],
       [2.2456806],
       [3.0294497],
       [2.4370682],
       [2.7587154],
       [2.1143892],
       [2.142542 ],
       [1.9081217],
       [2.7535613],
       [2.511846 ],


In [99]:
y_predict +=3

In [100]:
col_name=df2.columns.tolist()                   # 將數據框的列名全部提取出來存放在列表裏
print(col_name)

df2['Label']=y_predict   # 給city列賦值
print(df2)



['ID', 'Headline', 'Category', 'Label']
      ID                                           Headline     Category  \
0      1  Chelsea have no fit strikers ahead of trip to ...     football   
1      2  Microsoft opens up windows to Android and Appl...  sciencetech   
2      3  Fashion editor Meghan Blalock recounts her jou...       femail   
3      4  Three police officers accused of stealing ?? 3...         news   
4      5  As David Beckham shows off his beard, femail c...       femail   
..   ...                                                ...          ...   
222  223  Life on earth is flourishing and more diverse ...  sciencetech   
223  224  Ed balls refuses to match Tory election pledge...         news   
224  225  Nurse is paid ?? 2,200 for one 12-hour shift t...       health   
225  226              How London is being sold off to Qatar         news   
226  227  United Launch Alliance reveals plans for its r...  sciencetech   

        Label  
0    2.542996  
1    2.743884  

In [101]:
df2.to_csv('0751231.csv',index=0,columns = ['ID','Label'])

In [102]:
from keras.models import model_from_json

json_string = model.to_json() 
with open("model.config", "w") as text_file:    
    text_file.write(json_string)
model.save_weights("model.weight")
from keras.models import load_model
model.save('model.h5')  # creates a HDF5 file 'model.h5'